<a href="https://colab.research.google.com/github/Eddiebee/AI-Craft/blob/main/simple_LLM_application_using_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple LLM application using LangChain Expression Language (LCEL)

This notebook is aimed at showing us how to build a simple LLM application using LangChain. This application will translate text from English into another language.
It is just a simple LLM application that leverages an LLM call and some prompting.

A lot of features can be built out with just some prompting and an LLM call!

## Step 1: Installation

In [ ]:
!pip3 install -U langchain langchain-google-vertexai

We can safely ignore the above errors.

### Setup LangSmith
Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

In [ ]:
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "..."

## Step 2: Using Language Models

We'll use the Language Model by itself. LangChain supports many different language models. In this notebook we'l be using the models available on Google Vertex AI.

In [ ]:
# os.environ["GOOGLE_API_KEY"] = "..."

In [ ]:
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(model="gemini-pro")

Let's first use the model directly. The thing here is that `ChaModels` are instances of LangChain 'Runnables', which means that a expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to Italian"),
    HumanMessage(content="hi!")
]

model.invoke(messages)

AIMessage(content='Ciao! \n\nWhat would you like me to translate from English to Italian today? \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 9, 'candidates_token_count': 19, 'total_token_count': 28}}, id='run-dc03e715-b6e1-4e99-b47e-19c59dc19015-0')

## OutputParsers
Notice that the output from the model is `AIMessage`, which contains a string response as well as other metadata about the response. Most times we may just want to work with just the string reponse. We can do this by making use of a simple output parser.

We first import the simple string output parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

One possible way to use it is by itself. We can just pass the output of the model to it.

In [ ]:
response = model.invoke(messages)

In [ ]:
parser.invoke(response)

'Ciao!'

Most commonly we can "chain" the model with this output parser. This simply means that this output parser will be called everytime this chain is being executed. This chain takes on the input type of the language model (string or list message) and returns the output type of the output parser which in this case is a string.

We can easily create the chain using the `|` operator. The `|` operator is used to combine two elements together.

In [ ]:
chain = model | parser

In [ ]:
chain.invoke(messages)

"Ciao! 👋 \n\nWhat would you like me to translate from English to Italian? 😊 \n \nJust provide the text you want translated, and I'll do my best to give you an accurate and natural-sounding translation in Italian. 🇮🇹 \n"

## Prompt Template

Right now we're passing a list of messages directly into the language model. But, where are these messages coming from?
Usually, it is constructed from a combination of user's prompt and some application logic. This application logic usually takes the user input and then applies some transformations to it before then passing it to the language model. Common transformation would be taking the user input and adding a system message to it or formatting a template to include the user input.

`PromptTemplate`s are a concept in LangChain designed to take care of appying these transformations. They take in raw user input and return data (a prompt) that is ready to be passed into the language model.

Let's create a `PromptTemplate` here. It will take in two user variables:
- `language`: the language the text is to be translated into
- `text`: the text to be translated

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_message = "Translate the following into {language}:"

Next we are going to create the `PromptTemplate`. This will be a combination of the `system_template` as well as a simpler template for where to put the text.

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_message), ("user", "{text}")]
)

The input to this prompt template is a dictionary. Let's play around this by itself to see what is does by itself.

In [ ]:
result = prompt_template.invoke({"language": "italian", "text": "hi!"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi!')])

We can access the messages directly via:

In [ ]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi!')]

### Chaining together components with LCEL

We can now combine our above `ChatPromptTemplate` with the `model` and `parser` using the pipe (`|`) operator:

In [ ]:
chain = prompt_template | model | parser

In [ ]:
chain.invoke({"language": "italian", "text": "hi"})

'Ciao! Come posso aiutarti oggi?'

In [ ]:
chain.invoke({"language": "latin", "text": "hi"})

'Greetings to you as well! 👋\n\nWhat would you like me to translate into Latin today? 🤔\n'

In [ ]:
chain.invoke({"language": "latin", "text": "hi!"})

'Salve! 👋 \n\nWhat would you like me to translate into Latin today? 🤔 \n'

In [ ]:
chain.invoke({"language": "latin", "text": "hi"})

'Salve! 👋 How can I assist you today? 😊'

In [ ]:
chain.invoke({"language": "latin", "text": "peace"})

'pax'

In [ ]:
chain.invoke({"language": "latin", "text": "God"})

'Deus'

In [ ]:
chain.invoke({"language": "efik", "text": "Good"})

'I am unable to fulfill this request, as it is not in Efik but rather a greeting in English. If you provide me with the text you want translated, I will be happy to assist you. \n'

In [ ]:
chain.invoke({"language": "efik", "text": "Good morning"})

''

In [ ]:
chain.invoke({"language": "efik", "text": "Good"})

'The translation of "Good" in Efik is "Edi". \n\nIt can be used in various contexts, such as:\n\n* **As a greeting:** Edi ubok (Good morning), Edi osondo (Good afternoon), Edi abasi (Good evening)\n* **To express approval:** Edi ndom (Good work), Edi mfate (Good idea)\n* **To wish someone well:** Edi enyene (Good health), Edi usen (Good day)\n'

In [ ]:
chain.invoke({"language": "Nigerian pidgin", "text": "hi"})

'How you dey? \n'

This is indeed a simple example of the `LangChain Expression Language (LCEL)` to chain together LangChain modules. There several benefits to this approach of invoking LangChain modules such as optimized streaming and tracing support.

## Serving with LangServe

Now that we've built the application, we need to serve it. This is where `LangServe` comes in. Which is actually a package that helps developers deploy LangChain chains as REST APIs.

We'll be using Python Script for this.